# NNodely Documentation - State Variables

In [1]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-- nnodely_v1.5.0 --<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


## State Definition

Use the keyword 'State' to define a state variable (like you will do for an Input). You can specify the dimension of the variable.

In [2]:
clearNames('x_state')
x_state = Input('x_state', dimensions=1)
x_out = Fir(x_state.tw(0.5))

## Closed Loop

Every relation inside NNodely can update a state variable. closing a state in a loop means that at the end of each forward pass the result of the selected relation will update the selected state variable.

In [3]:
clearNames('out')
x_out.closedLoop(x_state)
out = Output('out',x_out)

or you can use the ClosedLoop block

In [4]:
clearNames('out')
x_out = ClosedLoop(x_out, x_state)
out = Output('out',x_out)

## Connect

Every relation inside NNodely can update a state variable. connecting a relation to a state means that at each forward pass the result of the selected relation will immediately update the selected state variable.

(Note: you must re-define the relation in order to change the update of the state variable)

In [5]:
clearNames()
x_out = Fir(x_state.tw(0.5))
x_out.connect(x_state)
out = Output('out',x_out)

or you can use the Connect block

In [6]:
clearNames('out')
x_out = Connect(x_out, x_state)
out = Output('out',x_out)

## Connect two models together

In [9]:
clearNames(['a','b_t','c','d_t','b_in','shared','b','A','B','C','D','d'])
import numpy as np

def linear_function(x, k1, k2):
    return x*k1 + k2

data_a = np.arange(1,101, dtype=np.float32)
data_b_t = linear_function(data_a, 2, 3)

data_c = np.arange(1,101, dtype=np.float32)
data_b_in = np.arange(5,105, dtype=np.float32)
data_d_t = linear_function(data_c, 5, 1)

dataset = {'a': data_a, 'b_t': data_b_t, 'c':data_c, 'b_in': data_b_in, 'd_t':data_d_t }
## Model a
a = Input('a')
b_t = Input('b_t')
shared = Parameter('shared',dimensions=(1,1))
output_relation = Linear(W=shared)(a.last())+Linear(W='A')(Fir(W='B')(a.tw(0.5)))
b = Output('b',output_relation)

model = Modely(seed=42)
model.addModel('b_model', b)
model.addMinimize('b_min', b, b_t.last())
model.neuralizeModel(0.1)

# Model d
c = Input('c')
d_t = Input('d_t')
b_in = Input('b_in')
output_relation.connect(b_in)
d = Output('d',Linear(W=shared)(c.last())+Fir(W='C')(c.tw(0.5))+Fir(W='D')(b_in.tw(0.3)))

model.addModel('d_model', [b,d])
model.addMinimize('d_min', d, d_t.last())
model.neuralizeModel(0.1)
model.loadData('dataset', dataset)

params = {'num_of_epochs': 1,
        'train_batch_size': 8,
        'val_batch_size': 8,
        'test_batch_size':1,
        'lr':0.1}

## training dei parametri di tutti i modelli
_ = model.trainModel(splits=[100,0,0], training_params=params, prediction_samples=4)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.1,
          'nnodely_version': '1.5.0',
          'ns': [5, 0],
          'ntot': 5,
          'num_parameters': 7},
 'Inputs': {'a': {'dim': 1,
                  'ns': [5, 0],
                  'ntot': 5,
                  'sw': [-1, 0],
                  'tw': [-0.5, 0]},
            'b_t': {'dim': 1, 'ns': [1, 0], 'ntot': 1, 'sw': [-1, 0]}},
 'Minimizers': {'b_min': {'A': 'Add47', 'B': 'SamplePart49', 'loss': 'mse'}},
 'Models': 'b_model',
 'Outputs': {'b': 'Add47'},
 'Parameters': {'A': {'dim': [1, 1], 'values': [[0.600895345211029]]},
                'B': {'dim': 1,
                      'tw': 0.5,
                      'values': [[0.8822692632675171],
                                 [0.9150039553642273],
                                 [0.38286375999450684],
                                 [0.9593056440353394],
                        

## Recurrent Train

In order to do a recurrent training of the network using the State variables is mandatory to specify the window of prediction (prediction_samples).

In [13]:
import numpy as np
clearNames(['x','x_state','y_state','out'])
x = Input('x', dimensions=3)
x_state = Input('x_state', dimensions=3)
y_state = Input('y_state', dimensions=3)
x_out = Linear(output_dimension=3)(x_state.tw(0.5))
y_out = Linear(output_dimension=3)(y_state.tw(0.5))
x_out.closedLoop(x_state)
y_out.closedLoop(y_state)
out = Output('out',x_out+y_out)

test = Modely(seed=42)
test.addModel('model', out)
test.addMinimize('error', out, x.tw(0.5))

test.neuralizeModel(0.1)

dataset = {'x':np.array([np.random.uniform(1,4,300)]).reshape(100,3).tolist()}
test.loadData(name='dataset', source=dataset)

# Training non ricorrente
params = {'num_of_epochs': 10, 'train_batch_size': 1, 'val_batch_size':1, 'lr':0.01}
tp = test.trainModel(splits=[70,20,10], prediction_samples=5, shuffle_data=False, training_params=params)
print('finale state: ', test._states)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.1,
          'nnodely_version': '1.5.0',
          'ns': [5, 0],
          'ntot': 5,
          'num_parameters': 18},
 'Inputs': {'x': {'dim': 3, 'ns': [5, 0], 'ntot': 5, 'tw': [-0.5, 0]},
            'x_state': {'closedLoop': 'Linear92',
                        'dim': 3,
                        'local': 1,
                        'ns': [5, 0],
                        'ntot': 5,
                        'tw': [-0.5, 0]},
            'y_state': {'closedLoop': 'Linear95',
                        'dim': 3,
                        'local': 1,
                        'ns': [5, 0],
                        'ntot': 5,
                        'tw': [-0.5, 0]}},
 'Minimizers': {'error': {'A': 'Add96', 'B': 'TimePart98', 'loss': 'mse'}},
 'Models': 'model',
 'Outputs': {'out': 'Add96'},
 'Parameters': {'PLinear75W': {'dim': [3, 3],
                        

## Clear State

use the specific function to manually clear the state of a state variable

In [14]:
test.resetStates()
print('finale state: ', test.states)

finale state:  {'y_state': [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]], 'x_state': [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]]}


## State update at training time

States variables can also be created at training time. These variables will exist only during the training process.

To create the you have to define a dictionary containing {input:output}. In that case, the 'input' will become a state during the training process and the 'output' will be the relation updating it.

In [15]:
import numpy as np
clearNames()
x = Input('x', dimensions=3)
x_s = Input('x_s', dimensions=3)
y_s = Input('y_s', dimensions=3)
x_out = Linear(output_dimension=3)(x_s.tw(0.5))
y_out = Linear(output_dimension=3)(y_s.tw(0.5))
out = Output('out',x_out+y_out)
out_x = Output('out_x',x_out)
out_y = Output('out_y',y_out)

test = Modely(seed=42)
test.addModel('model', [out,out_x,out_y])
test.addMinimize('error', out, x.tw(0.5))

test.neuralizeModel(0.1)

dataset = {'x':np.array([np.random.uniform(1,4,300)]).reshape(100,3).tolist()}
test.loadData(name='dataset', source=dataset)

# Training non ricorrente
params = {'num_of_epochs': 10, 'train_batch_size': 4, 'val_batch_size':4, 'test_batch_size':1, 'lr':0.01}
test.trainModel(splits=[70,20,10], prediction_samples=3, shuffle_data=False, closed_loop={'x_s':'out_x','y_s':'out_y'}, training_params=params)
print('finale state: ', test.states)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.1,
          'nnodely_version': '1.5.0',
          'ns': [5, 0],
          'ntot': 5,
          'num_parameters': 18},
 'Inputs': {'x': {'dim': 3, 'ns': [5, 0], 'ntot': 5, 'tw': [-0.5, 0]},
            'x_s': {'dim': 3, 'ns': [5, 0], 'ntot': 5, 'tw': [-0.5, 0]},
            'y_s': {'dim': 3, 'ns': [5, 0], 'ntot': 5, 'tw': [-0.5, 0]}},
 'Minimizers': {'error': {'A': 'Add105', 'B': 'TimePart107', 'loss': 'mse'}},
 'Models': 'model',
 'Outputs': {'out': 'Add105', 'out_x': 'Linear101', 'out_y': 'Linear104'},
 'Parameters': {'PLinear3W': {'dim': [3, 3],
                              'values': [[0.13318592309951782,
                                          0.9345980882644653,
                                          0.5935796499252319],
                                         [0.8694044351577759,
                                          0.56771528